# Email Subject Generator

We will use a LLM to generate an appropriate Email subject for a given email body.

In [3]:
import ollama
import requests
from IPython.display import display, Markdown

In [45]:
# LLM details
REST_OLLAMA_ENDPOINT = "http://localhost:11434/api/chat"
MODEL = "deepseek-r1:1.5b" # Thinking model, the thoughts are enclosed in <think> tags
OLLAMA_KEY = "ollama"

### Using REST Endpoint with local Ollama

In [23]:
# Instead of using the ollama package, we can also use the REST endpoints to do the operations.
messages = [{"role": "user", "content": "Hi, how are you?"}]

payload = {
    "model": MODEL,
    "messages": messages,
    "stream": False
}

response = requests.post(REST_OLLAMA_ENDPOINT,
                         json=payload,
                         headers={"Content-Type": "application/json"})

In [9]:
response.json()

{'model': 'deepseek-r1:1.5b',
 'created_at': '2025-04-01T07:13:31.1837912Z',
 'message': {'role': 'assistant',
  'content': "<think>\n\n</think>\n\nHello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today? 😊"},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 589407300,
 'load_duration': 29056300,
 'prompt_eval_count': 9,
 'prompt_eval_duration': 3632500,
 'eval_count': 44,
 'eval_duration': 556718500}

### Using Ollama library

In [13]:
# Using Ollama lib
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

<think>

</think>

Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today? 😊


In [39]:
# We can also use the openAI lib by just setting the endpoint of the local ollama server
from openai import OpenAI

messages = [{"role": "user", "content": "Hi, how are you?"}]

# Since ollama follows similar API path like OpenAI, hence the lib tries to read the endpoint - http://localhost:11434/v1/chat/completions
ollama_server = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
response = ollama_server.chat.completions.create(model=MODEL, messages=messages)

print(response.choices[0].message.content)

<think>

</think>

Hi! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How are *you* doing? 😊


In [66]:
def system_prompt():
    return """You are an expert writer. You will be given contents of an email and your task is to suggest the best possible subject for it.
    Try to create a subject which increases the likelihood of the reader reading it by reading the subject. The output should just be the subject line.
    """

def user_prompt(content: str):
    return f"""Here is an email body, suggest a subject for it. Email is as follows:
    {content}
    """

def create_messages(content: str):
    return [{"role": "system", "content": system_prompt()},
            {"role": "user", "content": user_prompt(content)}]

def suggest_email_subject(email: str):
    email_messages = create_messages(email)
    reponse = ollama_server.chat.completions.create(model="llama3.2:latest", messages=email_messages)
    
    return response.choices[0].message.content

In [69]:
email = """Hi <recepient>, I am currently working at XYZ and have 5+ YOE. I am looking for some engineering roles, please let me know if you have any open positions. 
Tech Stack - Python, C#, Kubernetes, MySQL, Distributed Systems, Microservices"""

subject = suggest_email_subject(email)
print(subject)

Based on the email body, a suitable subject could be:

"Job Opportunity Inquiry for Engineering Roles"

This subject accurately conveys the purpose of the email and highlights your interest in potential job openings.

Alternatively, you could also consider these other options:

* "Seeking Engineering Opportunities"
* "Request for Engineering Job Openings"
* "Career Update: Experiencing Increased YOE - Seeking New Opportunities"
* "Exploring Engineering Positions at XYZ"

Choose the one that best fits your tone and style!


In [70]:
display(Markdown(subject))

Based on the email body, a suitable subject could be:

"Job Opportunity Inquiry for Engineering Roles"

This subject accurately conveys the purpose of the email and highlights your interest in potential job openings.

Alternatively, you could also consider these other options:

* "Seeking Engineering Opportunities"
* "Request for Engineering Job Openings"
* "Career Update: Experiencing Increased YOE - Seeking New Opportunities"
* "Exploring Engineering Positions at XYZ"

Choose the one that best fits your tone and style!